In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import pyperclip

from scipy.interpolate import interp1d

from lared_laughter.constants import annot_exp_path, dataset_path
from utils import CovfeeParser, get_hit_stats, interp_30fps

# Annotation processing

This file processes the covfee outputs into a single dataframe that aggregates the results of the human annotation experiments.
An additional pkl file is produced with the continuous results

In [3]:
# open examples
laughter_examples_df = pd.read_csv(os.path.join(annot_exp_path, 'laughter_examples', 'examples_with_rect.csv'), index_col=0)
speech_examples_df = pd.read_csv(os.path.join(annot_exp_path, 'speech_examples', 'examples_with_rect.csv'), index_col=0)
calibration_examples_df = pd.read_csv(os.path.join(annot_exp_path, 'calibration_examples', 'examples.csv'))

In [4]:
laughter_examples_df['onset_time']     = laughter_examples_df['ini_time'] - laughter_examples_df['_ini_time']
laughter_examples_df['offset_time']    = laughter_examples_df['_end_time'] - laughter_examples_df['ini_time']
speech_examples_df['onset_time']       = speech_examples_df['ini_time'] - speech_examples_df['_ini_time']
speech_examples_df['offset_time']      = speech_examples_df['_end_time'] - speech_examples_df['ini_time']
calibration_examples_df['onset_time']  = calibration_examples_df['ini_time'] - calibration_examples_df['_ini_time']
calibration_examples_df['offset_time'] = calibration_examples_df['_end_time'] - calibration_examples_df['ini_time']

In [5]:
laughter_examples = {row['hash']: row.to_dict() for _, row in laughter_examples_df.iterrows()}
speech_examples = {row['hash']: row.to_dict() for _, row in speech_examples_df.iterrows()}
calibration_examples = {row['hash']: row.to_dict() for _, row in calibration_examples_df.iterrows()}
print((len(laughter_examples), len(speech_examples), len(calibration_examples)))

(1684, 785, 6)


In [6]:
results_path_1 = os.path.join(annot_exp_path, 'covfee1')
results_path_2 = os.path.join(annot_exp_path, 'covfee2')

In [7]:
# open covfee results
hits = json.load(open('../laughter2.covfee.json', 'rb'))
parser = CovfeeParser(laughter_examples, speech_examples, calibration_examples, hits)

In [8]:
all_results_1 = parser.parse_v1(results_path_1)
all_results_2 = parser.parse_v2(results_path_2)

json_res is None for file /mnt/c/Users/Jose/gdrive/data/lared_laughter/annotation_experiment_2/covfee1/4198c11729cea33268040a725998f16478a6564d4af091b2d40b84d727e35aaa/111_rating_0219ab2467ec6269e71c5445002aaece1e03a2ea4f65aedf1f4da0a75aeffbcb_video_1_0_0.json


In [9]:
all_results = {**all_results_1, **all_results_2}

In [10]:
hits_df = pd.DataFrame([get_hit_stats(hit) for hit in all_results.values()]).sort_values(by=['hit_group', 'hit_num'])

In [11]:
hits_df.head()

,id,hit_group,hit_num,cont,pressed,is_laughter,intensity_not_none,intensities,confidences,duration,rating
2,1_1d181a,0,1,84/84,53/84,53/84,84/84,13-11-11-09-06-03-00,03-09-07-07-08-15-35,43.000000,4.0
32,2_1d181a,0,1,84/84,55/84,56/84,84/84,17-19-09-05-04-02-00,02-02-08-06-06-13-47,45.516667,4.0
12,1_99c625,0,2,84/84,49/84,52/84,49/84,10-11-12-04-08-02-02,03-04-09-03-15-16-27,41.600000,4.0
25,1_ec3023,0,2,84/84,61/84,61/84,50/84,08-10-10-01-12-08-00,00-00-01-01-08-15-54,41.866667,5.0
26,1_eda1bf,0,3,84/84,56/84,55/84,84/84,03-10-21-15-02-03-01,03-00-01-44-24-11-01,93.783333,3.0


In [14]:
hits_df['annot_id'] = range(1, len(hits_df)+1)
hits_df['rating'] = hits_df['rating'].astype(float)
hits_df['duration'] = hits_df['duration'].astype(float)
hits_df['rating'].fillna(-1, inplace=True)
def format_rating(f):
    if f == -1: return '-'
    return str(int(f))+'/5'
    
pyperclip.copy(hits_df.to_latex(
    columns=['annot_id', 'hit_group', 'hit_num', 'pressed', 'intensities', 'confidences', 'duration', 'rating'], 
    header=['Annotator ID', 'G', 'N', '# positive', 'Intensity', 'Confidence', 'Time taken', 'Rating'],
    formatters={'rating': format_rating, 'duration': '{:.1f}'.format},
    index=False,
    label='tab:hit_details',
    caption='Details of the annotation HITs. \\textit{G} indicates the HIT group. HITs within the same group contain the same laughter/non-laughter samples. \\textit{N} indicates the HIT number within the group. HITs with the same \\textit{N} are identical, except for the (random) ordering of the samples within each condition. HITs with different \\textit{N} contain the same samples but assigned to different conditions. Each row corresponds to one annotator (HIT). \\textit{\# positive} indicates the number of times laughter was detected by this person. \\textit{Intensities} indicates the histogram of laughter intensities by each annotator (positive examples only). Each number (in order) corresponds to one step in the Likert scale (1-7).'
    ))

/tmp/ipykernel_2642/237448400.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pyperclip.copy(hits_df.to_latex(


## Interpolate

In [95]:
results = []
annotations = {}
for res in all_results.values():
    for ex_hash, example in res['tasks'].items():
        results.append(res['tasks'][ex_hash])

        try:
            continuous_data = res['continuous'][ex_hash]
        except KeyError:
            print(f'Error in instance {example["instance_id"]}')
            continue

        annot = interp_30fps(
            continuous_data,
            example_len=(example['_end_time']-example['_ini_time'])
        )
        annotations[(example['instance_id'], example['hash'], example['condition'])] = annot

In [110]:
examples = pd.DataFrame.from_dict(results)
calibration_examples = examples[examples['calibration']]
examples = examples[~examples['calibration']]
# create a new hash column containing a rating-specific hash
# calculated from the combination of hit and example hash

In [111]:
len(examples), len(annotations), len(examples[examples['has_continuous'] == True])

(3954, 4242, 3954)

In [117]:
def filter_examples_missing_modalities():
    computational_examples = []
    for hash in examples['hash'].unique():
        hash_examples = examples[examples['hash'] == hash]
        
        video_examples = hash_examples[hash_examples['condition'] == 'video']
        audio_examples = hash_examples[hash_examples['condition'] == 'audio']
        av_examples = hash_examples[hash_examples['condition'] == 'av']

        num_examples = min(len(video_examples), len(audio_examples), len(av_examples))

        computational_examples.append(video_examples.iloc[:num_examples, :])
        computational_examples.append(audio_examples.iloc[:num_examples, :])
        computational_examples.append(av_examples.iloc[:num_examples, :])
    return pd.concat(computational_examples)

In [118]:
computational_examples_df = filter_examples_missing_modalities()

In [121]:
out_path = os.path.join(annot_exp_path, 'processed')
computational_examples_df.to_csv(os.path.join(out_path, 'examples_without_calibration.csv'))
pickle.dump(annotations, open(os.path.join(out_path, 'continuous.pkl'), 'wb'))

# HIT information

In [103]:
pd.set_option('display.max_colwidth', 150)
feedback_df = pd.DataFrame(hit_info).transpose()
feedback_df['annot_id'] = range(1, len(hits_df)+1)
feedback_df['feedback'].fillna('-', inplace=True)
feedback_df['rating'] = feedback_df['rating'].astype(float)
feedback_df['duration'] = feedback_df['duration'].astype(float)
feedback_df['rating'].fillna(-1, inplace=True)

In [104]:
feedback_df.tail(10)

,duration,rating,feedback,annot_id
2_7e49a08cdbcac9a9ad7658116b8a160dfa526831777981919e31172426ecc669,72.150000,4.0,-,39
2_8bfd003817f3348a53d7292c84d0a155c4ec793a5c3ef2dfe2a5f6e0a272d505,41.950000,-1.0,-,40
2_96bc549cc2a7b0e3df2888482f5199982b80d32254f53df9f4684309d9181403,32.233333,5.0,The tools were working perfectly and the process was smooth.,41
2_bd45ce15933da0e06d4d544f27a95cb5de3147aaf592557feb5bae9fa0a5d583,39.400000,4.0,At the beginning was a little bit complicated.,42
2_be99f777836ba4bffac2e20cc4e75a717d766fd5d2ccbe236dc1147d579b490b,54.566667,5.0,It was interesting to see how much difficult was to recognize the laughs of people when it was only video. Im confident a higher quality on it wou...,43
2_c9c20f2eee48c8f1f8120bb25055b3bbb167d1f9284f31b9ee303f548f2da9bd,73.150000,4.0,I just found it a bit long but interesting and I did not have an issue with the tool.,44
2_da567d02513f82ff3b0fe0c5d6237e4226ffadf44f1d9897a2efea683c0479ff,35.483333,-1.0,-,45
2_dc19285db2b452c1d3a7d625dcf0f302ee9e2eb3f690bddcddaf7ae22e58335f,40.050000,5.0,It was a kinda long but not too much.,46
2_eda1bf4c0b1962f12a7a92dbb3159f85887d48334e764f225aeedee9b4b7db16,33.400000,5.0,-,47
2_ff9b4ba9e0d4e962b5a1c5f584409289edb280057543ca3c98cedf9a1d31b0eb,60.266667,5.0,"It was indeed a unique, one-of-a-kind research, although being a bit long, I felt alright generally. A ""break"" of some sort in the middle would be...",48


In [109]:
pyperclip.copy(
    feedback_df.to_latex(
    columns=['annot_id', 'duration', 'rating', 'feedback'], 
    header=['Annotator ID', 'Time taken (s)', 'Rating', 'Feedback'],
    # float_format="{:.1f}".format,
    formatters={'rating': format_rating, 'duration': '{:.1f}'.format},
    column_format='',
    index=False,
    label='tab:hit_feedback',
    caption='Feedback given by annotators after completing the experiment.'
    ))

/tmp/ipykernel_1949/3810216713.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  feedback_df.to_latex(
